Adicionando o banco

In [ ]:
import pandas as pd
import numpy as np

# Carregar o CSV 
raw_path = "kendo_matches_work.csv"
df = pd.read_csv(raw_path)

print("Shape original:", df.shape)
display(df.head())

# Remover coluna-índice exportada por engano
idx_cols = [c for c in df.columns if c.lower().startswith("unnamed")]
if idx_cols:
    df = df.drop(columns=idx_cols)
    print(f"Removidas colunas de índice exportado: {idx_cols}")

# Padronização de tipos / mapeamento de valores
# Colunas esperadas
expected_cols = [
    "match_id", "ippon_number", "seconds_between", "ippon_taken",
    "men", "kote", "do", "tsuki"
]

# Verificar se todas existem (se faltar, acuse para evitar erro silencioso)
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colunas ausentes no CSV: {missing}")

# Remover espaços em branco em strings
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()

# Mapeamento de strings comuns para binário
# (O/o -> 0, I/l/| -> 1, yes/true/y -> 1, no/false/n -> 0)
binary_map = {
    "o": 0, "O": 0, "0": 0, "no": 0, "false": 0, "n": 0,
    "i": 1, "I": 1, "l": 1, "|": 1, "1": 1, "yes": 1, "true": 1, "y": 1
}

bin_cols = ["men", "kote", "do", "tsuki", "ippon_taken"]

for col in bin_cols:
    # aplica mapeamento em valores de texto conhecidos
    df[col] = df[col].replace(binary_map)
    # converte para numérico (coerce -> valores inválidos viram NaN)
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ippon_number deve ser inteiro (ex.: 1º ippon, 2º ippon...)
df["ippon_number"] = pd.to_numeric(df["ippon_number"], errors="coerce", downcast="integer")

# Tempo entre ippons: inteiro
df["seconds_between"] = pd.to_numeric(df["seconds_between"], errors="coerce", downcast="integer")

# match_id: inteiro
df["match_id"] = pd.to_numeric(df["match_id"], errors="coerce", downcast="integer")

# Validar campos binários (0/1)
# Qualquer valor que não seja 0 ou 1 nas colunas binárias será considerado erro -> NaN
for col in bin_cols:
    df.loc[~df[col].isin([0, 1]), col] = np.nan

# Tratar tempos irregulares
# Regras:
#  - seconds_between deve ser >= 0
#  - valores absurdos ou sentinelas (ex.: 999) viram NaN
#  - por segurança, vamos considerar "absurdo" > 300s (5 min) neste contexto de rally entre golpes
df.loc[df["seconds_between"] < 0, "seconds_between"] = np.nan
df.loc[df["seconds_between"] > 300, "seconds_between"] = np.nan
# Se souber que 999 é sentinela de "desconhecido", já vira NaN automaticamente pela regra >300

# Descartar tuplas inconsistentes (conforme enunciado)
before_drop = df.shape[0]

# Regras de descarte:
#  a) campos críticos ausentes
critical = ["match_id", "ippon_number", "seconds_between", "ippon_taken", "men", "kote", "do", "tsuki"]
df = df.dropna(subset=critical)

#  b) ippon_number deve ser inteiro >= 1
df = df[df["ippon_number"] >= 1]

#  c) ippon_taken deve ser 0 ou 1 (já garantido acima, pois NaN foi removido)
#  d) técnicas (men/kote/do/tsuki) devem ser 0 ou 1 (também garantido acima)
#  e) opcional: se ninguém tentou golpe (tudo 0) e mesmo assim ippon_taken == 1, é inconsistente
mask_incoerente = (df[["men", "kote", "do", "tsuki"]].sum(axis=1) == 0) & (df["ippon_taken"] == 1)
df = df[~mask_incoerente]

after_drop = df.shape[0]
print(f"Removidas {before_drop - after_drop} tuplas por inconsistência/valores ausentes.")

# Remover duplicatas
# Se por acaso houver linhas idênticas (mesmo match e mesmos valores), manter só a primeira
dup_before = df.shape[0]
df = df.drop_duplicates()
dup_after = df.shape[0]
print(f"Removidas {dup_before - dup_after} tuplas duplicadas exatas.")

# Também é comum garantir unicidade por (match_id, ippon_number).
# Se existir mais de uma linha para o mesmo (match_id, ippon_number), manter a primeira.
grp_before = df.shape[0]
df = df.sort_values(["match_id", "ippon_number"]).drop_duplicates(subset=["match_id", "ippon_number"], keep="first")
grp_after = df.shape[0]
print(f"Após deduplicar por (match_id, ippon_number), removidas {grp_before - grp_after} linhas.")

# Estatísticas rápidas de checagem
print("\n--- Estatísticas ---")
print("Shape final:", df.shape)
print("\nContagem de ippon_taken:")
print(df["ippon_taken"].value_counts(dropna=False))

print("\nValue counts por golpe:")
for col in ["men", "kote", "do", "tsuki"]:
    print(f"\n{col}:\n", df[col].value_counts(dropna=False))

print("\nseconds_between (descrição):")
display(df["seconds_between"].describe())

# Salvar CSV limpo 
clean_path = "kendo_matches_clean.csv"
df.to_csv(clean_path, index=False)
print(f"\nArquivo salvo em: {clean_path}")

Shape original: (202, 9)


,Unnamed: 0,match_id,ippon_number,seconds_between,ippon_taken,men,kote,do,tsuki
0,1,1,1,1,0,1,0,0,0
1,2,1,2,5,1,0,1,0,0
2,3,2,1,10,0,0,1,0,0
3,4,2,2,48,0,1,0,0,0
4,5,3,1,38,0,1,0,0,0


Removidas colunas de índice exportado: ['Unnamed: 0']
Removidas 2 tuplas por inconsistência/valores ausentes.
Removidas 1 tuplas duplicadas exatas.
Após deduplicar por (match_id, ippon_number), removidas 0 linhas.

--- Estatísticas ---
Shape final: (199, 8)

Contagem de ippon_taken:
ippon_taken
0.0    112
1.0     87
Name: count, dtype: int64

Value counts por golpe:

men:
 men
0.0    106
1.0     93
Name: count, dtype: int64

kote:
 kote
0.0    137
1.0     62
Name: count, dtype: int64

do:
 do
0.0    166
1.0     33
Name: count, dtype: int64

tsuki:
 tsuki
0.0    189
1.0     10
Name: count, dtype: int64

seconds_between (descrição):


C:\Users\biels\AppData\Local\Temp\ipykernel_14840\3941513205.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(binary_map)


count    199.000000
mean      25.723618
std       13.752832
min        1.000000
25%       12.500000
50%       26.000000
75%       37.000000
max       50.000000
Name: seconds_between, dtype: float64


Arquivo salvo em: kendo_matches_clean.csv
